In [1]:
from datautil.common import path_to_save_base
from datautil.gamelog import result_parse

In [2]:
import re
def parse(logs):
    cursor = 0
    inning = (0, 0)
    pitcher_id = None 
    batter_id = None

    watch = (0,0)

    stack = []
    last_result = None
    
    results = []
    
    while cursor < len(logs):
        ln = logs[cursor]

        res_inn = re.search("(Top|Bottom) of the ([0-9]+)(st|nd|rd|th) -", ln)
        if res_inn != None:
            inning = (int(res_inn.group(2)), 0 if res_inn.group(1)=='Top' else 1)
            cursor += 1
            continue

        res_p = re.search("Pitching: .HP <a href=\"../players/player_(\d+).html", ln)
        if res_p != None:
            pitcher_id = int(res_p.group(1))
            #print ("Pitcher", pitcher_id, "for home team" if inning[1]=="Top" else "for road team")
            cursor += 1
            continue

        # Pinch Hitting: LHB <a href="../players/player_543.html">Morgan McCants</a>
        res_b = re.search("(Batting|Pinch Hitting): .HB <a href=\"../players/player_(\d+).html", ln)
        if res_b != None:
            if len(stack) > 0 and last_result == None:
                print(inning)
                print("".join(["*** " + a for a in stack]))
                # raise "Look"

            batter_id = int(res_b.group(2))
            #print ("- Batter", batter_id, "PH" if res_b.group(1) == "Pinch Hitting" else "")
            cursor += 1
            stack = []
            results.append({'result':last_result, 'batter':batter_id, 'pitcher':pitcher_id})
            last_result = None
            continue

        evntList = [
            ("OUT", ["GO"], "Ground out ([^ ]*) \((.*), (.*), EV (.*) MPH\)", ["def", "gftype", "loc", "spd"]),
            ("OUT", ["GO"], "Grounds out, ([^ ]*) \((.*), (.*), EV (.*) MPH\)", ["def", "gftype", "loc", "spd"]),
            # TODO: resolve flyout
            ("OUT", ["FO"], "Fly out, ([^ ]*) *\((.*), (.*), EV (.*) MPH\)", ["def", "gftype", "loc", "spd"]),
            ("OUT", ["GO", "GIDP"], "Grounds into (double|DOUBLE) play, ([^ ]*) *\((.*), (.*), EV (.*) MPH\)", ["_", "def","gftype","loc","spd"]),
            ("OUT", ["TRIPLE"], "Lines into TRIPLE play, (.*) ([^ ]*) *\((.*), (.*), EV (.*) MPH\)", ["def","gftype","loc","spd"]),
            ("OUT", ["FC"], "Fielders Choice at .*, (.*) \((.*), (.*), EV (.*) MPH\)", ["def","gftype","loc","spd"]),
            ("OUT", ["FC"], "Grounds into fielders choice (.*) \((.*), (.*), EV (.*) MPH\)", ["def","gftype","loc","spd"]),
            ("FC", [], "Fielders Choice attempt at .*, Runner SAFE", []),
            ("HIT", [], "<b>(SINGLE|DOUBLE|TRIPLE)</b>  \((.*), (.*), EV (.*) MPH\)", ["hittype", "gftype","loc","spd"]),
            ("HR", [], "<b>([1-9A-Z-]*|GRAND SLAM) HOME RUN</b>  \((.*), (.*), EV (.*) MPH\)", ["hrtype", "gftype","loc","spd"]),
            ("OUT", ["SO"], "Strikes out (looking|swinging)", ["sotype"]),
            ("ERR", ["DEF"], "Reached on error, E(.) \((.*), (.*), EV (.*) MPH\)", ["def", "gftype","loc","spd"]),
            ("ERR", ["DROP"], "Reached via error on a dropped throw from (.*), E(.) \((.*), (.*), EV (.*) MPH\)", ["def", "culprit", "gftype","loc","spd"]),
            ("ERR", ["CI"], "Reaches on Catchers interference", []),
            ("HBP", [], "Hit by Pitch", []),
            ("BB", [], "Base on Balls", []),
            ("BB", [], "3-[012]: Ball", []),
            ("IBB", [], "Intentional Walk", []),
            ("HIT", ["ERR"], "Single, Error in .*, E(.), batter to second base \((.*), (.*), EV (.*) MPH\)", ["culprit","gftype","loc","spd" ]),
            ("OUT", ["HIT", "RUNDOWN"], "SINGLE, rundown, out", []),
            ("HIT", [], "SINGLE, rundown, safe", []),
            ("HIT", ["OUT"], "SINGLE, but batter called out on appeal for missing .* base!", []),
            # Bunts
            ("OUT", ["FO", "BUNTFAIL"], "Bunt - Flyout to ([^ ]*)", ["loc"]),
            ("OUT", ["SO", "BUNTFAIL"], "(Bunted foul|Bunt missed), Strikeout!", ["reason"]),
            ("OUT", ["SACBUNT"], "Sac Bunt to ([^ ]*)", ["loc"]),
            ("OUT", ["GO", "BUNTFAIL", "DP"], "Sac Bunt - play at second, runner OUT -> throw to first, DP!", []),
            ("HIT", ["BUNT"], "Bunt for hit to ([^ ]*) - play at first, batter safe!", ["loc"]),
            ("HIT", ["BUNT"], "Sac Bunt by (.*) - play at .*, runner safe!", ["loc"]),
            ("OUT", ["GO", "BUNTFAIL"], "Bunt for hit to (.*) - play at first, batter OUT! (.*)", ["loc", "def"]),
            ("HIT", ["BUNT"], "<b>Squeeze Bunt to (.*) - play .*, runner scores, batter safe!</b>", ["loc"]),
            ("FC", ["BUNT"], "Squeeze Bunt to (.*) - play .*, runner OUT, batter safe! .*", ["loc", "def"]),
            # not PA result
            ("RUNNEROUT", ["PICKOFF"], "Pickoff Throw to (First|Second|Third) - Out", ["base"]),
            ("RUNNEROUT", ["PICKOFF"], "Pickoff Play to (.*), Runner tries for .*, OUT after rundown.", ["base"]),
            ("RUNNEROUT", ["PICKOFF"], "Pickoff Throw by Catcher to (First|Second|Third) - Out!", ["base"]),
            ("RUNNEROUT", ["STEALFAIL"], "player_(\d+)\.html.*is caught stealing ([^ ]*) (base [^ ]*)", ["runner", "base", "def"]),
            ("RUNNEROUT", ["STEALFAIL"], "<b>Steal of home, .*player_(\d+)\.html.* is out</b>", ["runner"])
        ]

        for ev, basetags, regx, info in evntList:
            res = re.search(regx, ln)
            if res != None:
                try:
                    infomap = dict(zip(info, res.groups()))
                except Exception as e:
                    print (ev)
                    print (info)
                    print (*res.groups())
                    raise e

                tags = basetags
                if "DP" in ln and "DP" not in tags:
                    tags.append("DP")
                if "FO" in tags:
                    if "Runner from 3rd tags up, SCORES" in logs[cursor+1]:
                        tags.append("SF")
                #print ("-- {} {} {}".format(ev, tags, infomap))
                last_result = (ev, tags, infomap)

        stack.append(ln)

        cursor += 1
    
    return results


In [22]:
import os, time
def_res = {}

for test in range(8001, 8005):
    save = 'test_' + str(test)

    dn = path_to_save_base(save) + "/news/txt/leagues/"

    files = os.listdir(dn)
    start = time.time()
    for fn in files:
        try:
            for ev in parse(open(dn + fn).readlines()):
                # print (ev)
                if "loc" in ev[2]:
                    loc = ev[2].get('loc')
                    out, cnt = def_res.get(loc, (0,0))
                    cnt += 1
                    if ev[0] == 'OUT' or 'OUT' in ev[1]:
                        out += 1
                    def_res[loc] = (out, cnt)
        except Exception as e:
            print(fn)
            raise e
    print(time.time() - start)

40.694384813308716
(4, 1)
*** [%N]	Pickoff Throw to First - Error! E1
*** [%N]	<a href="../players/player_1297.html">Will Anfuso</a> to second

40.331310987472534
(4, 1)
*** [%N]	Balk!
*** [%N]	<a href="../players/player_1293.html">Bobby del Muro</a> to third
*** [%N]	<a href="../players/player_597.html">Dana Gaylord</a> to second

47.35461211204529
48.87969160079956


In [23]:
infield = 0
for loc, stat in sorted(def_res.items()):
    if loc[0] in "3456":
        infield += stat[1]

for loc, stat in sorted(def_res.items()):
    if loc[0] in "3456":
        print ("{},{:.2f}%,{:.2f}%".format(loc, stat[1]*100/infield, 100*stat[0]/stat[1]))

3,2.91%,98.10%
34,6.37%,84.51%
34D,6.87%,18.00%
34S,3.35%,86.43%
3D,2.04%,5.07%
3DF,0.02%,100.00%
3F,0.10%,100.00%
3L,1.85%,62.52%
3S,1.42%,85.97%
3S!,0.04%,100.00%
3SF,0.22%,100.00%
4,10.84%,96.02%
4D,2.78%,23.99%
4M,2.80%,73.59%
4MD,5.75%,15.12%
4MS,0.67%,64.86%
4S,1.58%,81.99%
5,2.46%,96.36%
56,8.43%,87.23%
56D,3.24%,15.47%
56S,4.12%,88.76%
5D,1.78%,8.66%
5DF,0.01%,100.00%
5F,0.07%,100.00%
5L,2.17%,45.69%
5S,1.43%,88.06%
5S!,0.03%,100.00%
5SF,0.16%,100.00%
6,13.15%,95.31%
6D,2.27%,20.71%
6M,2.93%,71.01%
6MD,5.78%,13.16%
6MS,0.74%,63.45%
6S,1.65%,80.93%
